In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db


'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


The 'facts' table we see above is the CIA world factbook. It contains a collection of demographic statistics about all the countries in the world. Let's check out the first five rows.

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The column headings seem to be straightforward and self explanatory. Let's begin by searching the table for outliers.

In [5]:
%%sql
SELECT MIN(population), MAX(population), 
       MIN (population_growth), MAX(population_growth)
  FROM facts;
                                                                      

Done.


MIN(population),MAX(population),MIN (population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Below are queries to find out what nations have these values.

In [8]:
%%sql
  SELECT name, population
    FROM facts
ORDER BY population
   LIMIT 10


Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None
Bouvet Island,None
Jan Mayen,None
British Indian Ocean Territory,None
South Georgia and South Sandwich Islands,None
Navassa Island,None


%%sql
  SELECT name, population
    FROM facts
ORDER BY population DESC
   LIMIT 10

We see there are several countries with no population and there are rows included for the whole world and the european union in the dataset. Now I will recalculate the summary statistics excluding that row.

In [16]:
%%sql
SELECT MIN(population), MAX(population), 
       MIN (population_growth), MAX(population_growth)
  FROM facts
 WHERE name NOT IN ('World', 'European Union');
                   

Done.


MIN(population),MAX(population),MIN (population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


Now moving on to average population and areas for the table

In [17]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts
 WHERE name NOT IN ('World', 'European Union');

Done.


AVG(population),AVG(area)
30235554.991666667,539893.1895161291


Below we will find all countries whose population is above average

In [18]:
%%sql
  SELECT name, population
    FROM facts
   WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name NOT IN ('World', 'European Union'))
         AND name NOT IN ('World', 'European Union')
ORDER BY population DESC

Done.


name,population
China,1367485388
India,1251695584
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745
Russia,142423773
Japan,126919659


Below are all countries whose area is below average

In [19]:
%%sql
  SELECT name, area
    FROM facts
   WHERE area <(SELECT AVG(area)
                       FROM facts
                      WHERE name NOT IN ('World', 'European Union'))
         AND name NOT IN ('World', 'European Union')
ORDER BY area 

Done.


name,area
Holy See (Vatican City),0
Monaco,2
Coral Sea Islands,3
Ashmore and Cartier Islands,5
Navassa Island,5
Spratly Islands,5
Clipperton Island,6
Gibraltar,6
Wake Island,6
Paracel Islands,7


Which countries are most densely populated?

In [20]:
%%sql
SELECT name, population, area, population/area AS density
FROM facts
WHERE name NOT IN ('European Union', 'World');
ORDER BY density DESC


(sqlite3.OperationalError) near "WHERE": syntax error
[SQL: SELECT name, population, area, population/area AS density
FROM facts
ORDER BY density DESC
WHERE name NOT IN ('European Union', 'World');]
(Background on this error at: http://sqlalche.me/e/13/e3q8)
